## **Analizador sintáctico predictivo ascendente**
Juan David Sánchez Murcia\
Maria Paula Vizcaíno Forero

Importamos librerías correspondientes

In [1]:
pip install anytree

In [2]:
import pandas as pd
from collections import deque
from anytree import Node, RenderTree, util
from anytree.exporter import UniqueDotExporter

Definimos la estructura correspondiente a nuestras terminales. Teniendo en cuenta las no terminales en mayuscula y la siguiente notación para las producciones la cual se indica con una flecha -> Grammar:

In [3]:
terminals=['(',')',';','x']
non_terminals=['S','A','C','B']
start_symbol='S'
grammar=['S->(A)',  
         'A->CB', 
         'B->;A|@', 
         'C->x|S']
data={'(':['S→(A)','A→CB','','C→S'],
      ')':['','','B→@',''],
      ';':['','','B→;A',''],
      'x':['','A→CB','','C→x'],
      '$':['','','','']}

productions_=[s.split('->') for s in grammar]
productions=dict([pair[0],pair[1].split('|')] for pair in productions_)

Definimos ahora *first* y *find_first* funciones las cuales corresponden a PRIMERO como su nombre lo indica

In [4]:
def first(alpha):
    first_set=set()
    if alpha in terminals or alpha =='@':
        first_set.add(alpha)
    elif alpha in non_terminals:
        alpha_prods=productions[alpha]
        for prod in alpha_prods: #Si hay más de una producción para un no terminal
            first_set_=first(prod)
            first_set.update(first_set_)
    else: #Es una producción del tipo alpha->y1y2...yk
        i=0
        y=alpha[0]
        y_first=first(y)
        if '@' in y_first:    #Encontre un epsilon en la primera, entonces calculo la siguiente
            i=1
            while '@' in y_first: #Calculo todas las y's hasta que en alguna salga un epsilon
               first_set.update(y_first)
               first_set.remove('@')
               first_set.update(alpha[i])
               i+=1
            if i == len(alpha):   #Epsilon aparece en todos los primeros
                first_set.add('@')
        else:
            first_set.update(y_first)
    return first_set

def find_first():
    first_={non_term:first(non_term) for non_term in non_terminals}
    return first_

Corremos *find_first*

In [5]:
find_first()

{'A': {'(', 'x'}, 'B': {';', '@'}, 'C': {'(', 'x'}, 'S': {'('}}

Análogamente, definimos ahora *find_follow* función la cual corresponde a SIGUIENTE como su nombre lo indica

In [6]:
def find_follow():
    follow={}
    for nonterminal in non_terminals:
        follow[nonterminal]=set()   
    follow[start_symbol].add('$')
    change=True
    while(change):
        change=False
        for non_term,prods in productions.items():
            for prod in prods:
                # nonterminal='A'
                for alpha in non_terminals:
                    if alpha in prod:
                        alpha_pos=prod.index(alpha)
                        if alpha_pos<len(prod)-1:  #Producción de la forma prod->a alpha beta
                            first_=first(prod[alpha_pos+1])
                            first_beta=first_
                            for i in range(alpha_pos+1,len(prod)):
                                first_=first(prod[i])  
                                first_beta=first_beta.intersection(first(prod[i]))
                            if not ((first_-set('@')).issubset(follow[alpha])): #Encontramos nuevos elementos para agregar
                                follow[alpha]=follow[alpha].union(first_-set('@'))
                                change=True
                            if not (prod[i] in non_terminals and '@' in first_beta) :
                                break
                            if '@' in first_beta:
                                if not (follow[non_term].issubset(follow[alpha])):
                                    follow[alpha]=follow[alpha].union(follow[non_term])
                                    change=True 
                        else: #Producción de la forma prod->a alpha 
                            if not (follow[non_term].issubset(follow[alpha])):
                                follow[alpha]=follow[alpha].union(follow[non_term])
                                change=True                                
    return follow

Corremos *find_follow*

In [7]:
find_follow()

{'A': {')'}, 'B': {')'}, 'C': {')', ';'}, 'S': {'$', ')', ';'}}

Continuamos con la generación de la tabla correspondiente a la gramática por medio de la función *parseTable*

In [8]:
def parseTable(grammar,terminals,non_terminals):
    terminals_=terminals
    terminals_.append('$')
    table_={non_term:{term:[] for term in terminals_ } for non_term in non_terminals}
    productions_=[s.split('->') for s in grammar]
    productions=dict([pair[0],pair[1].split('|')] for pair in productions_)
    follow_=find_follow()
    for A,prods in productions.items():
        for alpha in prods:
            first_alpha=first(alpha)
            for a in first_alpha:
                if a in terminals:
                    table_[A][a]='{} -> {}'.format(A,alpha)
                elif a == '@':
                    follow_A=follow_[A]
                    for b in follow_A:
                        if b == '$':
                            table_[A][b]='{} -> {}'.format(A,alpha)
                        table_[A][b]='{} -> {}'.format(A,alpha)
    return table_               


Corremos *parseTable*

In [9]:
info=parseTable(grammar,terminals,non_terminals)
df=pd.DataFrame.from_dict(info,orient='index')
df.loc['S']

(    S -> (A)
)          []
;          []
x          []
$          []
Name: S, dtype: object

In [10]:
stack=deque()

Continuamos con la definición de la funciones correspondientes a los factores de la pila simulándola con cadenas culminando con la función *parseStk* generando el árbol correspondiente

In [11]:
frame = pd.DataFrame(data, index=['S','A','B','C'])
root = Node(non_terminals[0])
stk = "$S"
ahead = ""
sub = ""

def nextToken():
    global sub
    if(sub[0:2] == "id"):
         sub = sub[2:]
         return "id"
    else:
        s = sub[0:1]
        sub = sub[1:]
        return s

def empty():
    if(len(stk) == 0):
        return True
    return False

def top():
    global stk
    if(stk[-1:] == '@'):
        return stk[-2:]
        print(stk)
    else:
        return stk[-1:]
        print(stk)

def pop():
    global stk
    if(stk[-1:] == '@'):
        stk = stk[:-2];
    else:
        stk = stk[:-1]

def push(s):
    global stk
    while s[-1:] != '→':
        if(s[-1:] == '@'):
            stk += s[-2:]
            s = s[0:-2]
            print(stk)
        else:
            stk += s[-1:]
            s = s[0:-1]
            print(stk)

def parseStk(top, head):
    global ahead
    if top == head :
        if head != '$':
            print('Match', head)
        ahead = nextToken()
        pop()
    else:
        s = frame[head][top]
        if s == '':
            ahead = nextToken()
        elif s == 'x':
            pop()
        else:
            print(s)
            a = s.split('→')
            for caracter in non_terminals:
              if a[0] == root:
                for caracter_ in a[1]:
                  Node(caracter_, root = a[0]) 
            pop()
            if(s[-1:] != '@'):
                push(s)

Corremos *parseStk* por medio de la pila y la tabla de producciones con la entrada (x;(x))

In [12]:
sub = '(x;(x))$'
sub += '$'
ahead = nextToken()
while(empty() == False):
    t = top()
    parseStk(t, ahead)

S→(A)
$)
$)A
$)A(
Match (
A→CB
$)B
$)BC
C→x
$)Bx
Match x
B→;A
$)A
$)A;
Match ;
A→CB
$)B
$)BC
C→S
$)BS
S→(A)
$)B)
$)B)A
$)B)A(
Match (
A→CB
$)B)B
$)B)BC
C→x
$)B)Bx
Match x
B→@
Match )
B→@
Match )


Teniendo en cuenta lo anterior generamos la función *parseTree*

In [13]:
input = ['(','x',';','(','x',')',')']

def parseTree():
  stk_ = ["$"]
  stk_+non_terminals[0]
  inputAct = 0
  a = input[inputAct]
  X = stk_[-1]
  root = Node(X)
  padre = root
  while(X != "$"):
    if X == a:
      for hijo in padre.children:
        if hijo.name == X:
          padre = hijo
      while(util.leftsibling(padre) == None and padre != root):
          padre = padre.parent
      if padre != root:
        padre = padre.parent
      stk_.pop(-1)
      inputAct += 1
      a = input[inputAct]
    elif X in terminals and X != "€":
      print("Error, ", X, "es un terminal")
      return
    elif X == "€":
      for hijo in padre.children:
        if hijo.name == X:
          padre = hijo
      while(util.leftsibling(padre) == None and padre != root):
          padre = padre.parent
      if padre != root:
        padre = padre.parent
      stk_.pop(-1)
    elif X in frame:
      if padre != None:
        for hijo in padre.children:
          if hijo.name == stk_[-1]:
            padre = hijo
            break
      stk_.pop(-1)
      terminalAct = ""
      for caracter in frame[X][a][::-1]:
        if caracter in non_terminals:
          if padre != None:
            Node(caracter, parent=padre)
          stk_.append(caracter)  
        else:        
          terminalAct += caracter
        if terminalAct[::-1] in terminals: 
          if padre != None:
            Node(terminalAct[::-1], parent=padre)
          stk_.append(terminalActua[::-1])
          terminalAct = ""
    X = stk_[-1]
  print(root.children)